# Notebook to import Sonata-based files into the database
![](../bioexplorer_vasculature_banner.png)

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm
import numpy as np
import h5py
import json
import requests
import glob
import assimp
from bioexplorer import Quaternion

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'atlas'
atlas_data_folder = os.getenv('ATLAS_DATA_FOLDER')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
data_folder = os.path.joint(atlas_data_folder, 'mouse', 'v2')

## Nodes

In [ ]:
f = h5py.File(os.path.join(data_folder, 'cell_atlas.h5'))
print(f.keys())
pxs = f['x']
pys = f['y']
pzs = f['z']

rxs = f['orientx']
rys = f['orienty']
rzs = f['orientz']

regions = f['regions']
etypes = f['eTypes']
cell_types = f['cellTypes']

In [ ]:
e_types = set(etypes)

In [ ]:
# e-types
print('Deleting existing regions')
with Session(engine) as session:
    session.execute('DELETE FROM %s.etype' % db_schema)
    session.commit()

with Session(engine) as session:
    for i in tqdm(range(len(e_types))):
        try:
            session.execute(
                'INSERT INTO %s.etype VALUES (:guid, :description)' % db_schema,
                {
                    'guid': int(list(e_types)[i]),
                    'description': ''
                }
            )
        except Exception as e:
            print(e)
            session.rollback()
            break
    session.commit()

In [15]:
# Regions
f = open(os.path.join(data_folder, 'brain_regions.json'))
obj = json.load(f)
update_only = False

def insert_region(session, node, level):
    guid = node['id']
    acronym = node['acronym']
    name = node['name']
    atlas_guid = node['atlas_id']
    ontology_guid = node['onthology_id']
    color_hex_triplet = node['color_hex_triplet']
    graph_order = node['graph_order']
    hemisphere_guid = node['hemisphere_guid']
    parent_guid = node['parent_structure_id']
    if not parent_id:
        parent_id = -1
    session.execute(
        'INSERT INTO %s.region VALUES (:guid, :code, :description, :parent_guid, :level, :atlas_guid, :ontology_guid, :color_hex_triplet, :graph_order, :hemisphere_guid)' % db_schema,
        {
            'guid': guid,
            'code': acronym,
            'description': name,
            'parent_guid': parent_guid,
            'level': level,
            'atlas_guid': atlas_guid,
            'ontology_guid': ontology_guid,
            'color_hex_triplet': color_hex_triplet,
            'graph_order': graph_order,
            'hemisphere_guid': hemisphere_guid
        }
    )
    for child in node['children']:
        insert_region(session, child, level + 1)
    session.commit()

def update_region(session, node, level):
    guid = node['id']
    name = node['name']
    atlas_guid = node['atlas_id']
    ontology_guid = node['ontology_id']
    color_hex_triplet = node['color_hex_triplet']
    graph_order = node['graph_order']
    hemisphere_guid = node['hemisphere_id']
    parent_guid = node['parent_structure_id']
    if not parent_guid:
        parent_guid = -1
    if not atlas_guid:
        atlas_guid = -1
    session.execute(
        'UPDATE %s.region SET description=:name, parent_guid=parent_guid, level=:level, atlas_guid=:atlas_guid, ontology_guid=:ontology_guid, color_hex_triplet=:color_hex_triplet, graph_order=:graph_order, hemisphere_guid=:hemisphere_guid WHERE guid=:guid' % db_schema,
        {
            'name': name,
            'parent_guid': parent_guid,
            'level': level,
            'atlas_guid': atlas_guid,
            'ontology_guid': ontology_guid,
            'color_hex_triplet': color_hex_triplet,
            'graph_order': graph_order,
            'hemisphere_guid': hemisphere_guid,
            'guid': guid
        }
    )
    for child in node['children']:
        update_region(session, child, level + 1)
    session.commit()

if update_only:
    print('Updating existing regions')
    with Session(engine) as session:
        for node in obj['msg']:
            update_region(session, node, 0)
        session.commit()
else:
    print('Deleting existing regions')
    with Session(engine) as session:
        session.execute('DELETE FROM %s.region' % db_schema)
        session.commit()


    with Session(engine) as session:
        for node in obj['msg']:
            insert_region(session, node, 0)
        session.commit()

f.close()

In [ ]:
# Cells
def euler_to_quaternion(yaw, pitch, roll):
    import numpy as np
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return Quaternion(qx, qy, qz, qw)

print('Deleting existing cell')
with Session(engine) as session:
    session.execute('DELETE FROM %s.cell' % db_schema)
    session.commit()

print('Populating cells')
with Session(engine) as session:
    for i in tqdm(range(83500001, pxs.size)):
        try:
            q = euler_to_quaternion(rxs[i], rys[i], rzs[i])
            etype = -1
            if i < etypes.size:
                etype = int(etypes[i])
            session.execute(
                'INSERT INTO %s.cell VALUES (:guid, :cell_type_guid, :region_guid, :electrical_type_guid, :x, :y, :z, :rotation_x, :rotation_y, :rotation_z, :rotation_w)' % db_schema,
                {
                    'guid': i,
                    'cell_type_guid': int(cell_types[i]),
                    'region_guid': int(regions[i]),
                    'electrical_type_guid': etype,
                    'x': float(pxs[i]),
                    'y': float(pys[i]),
                    'z': float(pzs[i]),
                    'rotation_x': float(q.x),
                    'rotation_y': float(q.y),
                    'rotation_z': float(q.z),
                    'rotation_w': float(q.w),
                }
            )
            if i % 500000 == 0:
                session.commit()
        except Exception as e:
            print(e)
            session.rollback()
            break
    session.commit()


## Meshes

In [ ]:
mesh_folder = os.path.joint(atlas_data_folder, 'mouse', 'meshes')

### Download meshes from Blue Brain Project

In [ ]:
with Session(engine) as session:
    data = session.execute('SELECT guid FROM %s.region' % db_schema)
    for guid in data.all():
        id = guid[0]
        base_url = 'https://bbpteam.epfl.ch/project/user/rodarie/CA/COMDATA1/meshesMS/'
        r = requests.get(base_url + 'decimated_smoothed_mesh_%d.obj' % id)
        f = open(os.path.join(mesh_folder, '%d.obj' % id), 'w')
        content = str(r.content).replace('\\n', '\n')
        f.write(content)
        f.close()

### Import meshes into DB

In [ ]:
def get_id_from_name(name):
    basename = os.path.basename(name).split('.')[0]
    return int(basename.split('_')[-1:][0])

print('Deleting existing meshes')
with Session(engine) as session:
    session.execute('DELETE FROM %s.mesh' % db_schema)
    session.commit()

files = glob.glob(os.path.join(mesh_folder, '*.obj'))
with Session(engine) as session:
    for i in tqdm(range(len(files))):
        filename=files[i]
        guid = get_id_from_name(filename)
        model = assimp.load(filename)
        if not model.meshes:
            continue
        mesh = model.meshes[0]
        try:
            session.execute(
                'INSERT INTO %s.mesh VALUES (:guid, :vertices, :indices, :normals, :colors)' % db_schema,
                {
                    'guid': guid,
                    'vertices': np.array(mesh.vertices, dtype=np.float32).tobytes(),
                    'indices': np.array(mesh.faces, dtype=np.uint32).tobytes(),
                    'normals': np.array(mesh.normals, dtype=np.float32).tobytes(),
                    'colors': np.array(mesh.colors, dtype=np.float32).tobytes()
                }
            )
        except Exception as e:
            print(e)
            session.rollback()
            break
        session.commit()    